In [1]:
import numpy as np
import cv2
from skimage.draw import line
import warnings
from scipy import ndimage, spatial
from scipy.ndimage import filters, morphology
warnings.filterwarnings("ignore")

In [2]:
def page_border(img):
    
    # Blur the image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gaussian_blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    thresh = cv2.threshold(
        gaussian_blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

    # Find contours and sort for largest contour
    cnts = cv2.findContours(
        thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

    error = True
    use_cnt = None
    display_cnt = None
    no_border = False

    # Sorted by area, find the largest contour which is approximately rectangular
    # Crop to this largest approximate rectangle
    for c in cnts:
        # Perform contour approximation
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)

        # Check if a rectangle
        if len(approx) == 4:
            error = False
            display_cnt = approx.reshape(4, 2)

            x1 = display_cnt[0][0] if (display_cnt[0][0] > display_cnt[1][0]) else display_cnt[1][0]
            y1 = display_cnt[0][1] if (display_cnt[0][1] > display_cnt[3][1]) else display_cnt[3][1]

            x2 = display_cnt[2][0] if (display_cnt[2][0] < display_cnt[3][0]) else display_cnt[3][0]
            y2 = display_cnt[1][1] if (display_cnt[1][1] < display_cnt[2][1]) else display_cnt[2][1]

            h, w = img.shape[:2]
            x1 = max(0, x1)
            y1 = max(0, y1)
            x2 = min(w, x2)
            y2 = min(h, y2)
            img = img[y1: y2, x1: x2]
            break

    imggray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imfft = np.fft.fft2(imggray)
    imffts = np.fft.fftshift(imfft)
    mags = np.abs(imffts)
    angles = np.angle(imffts)
    visual = np.log(mags)
    visual3 = np.abs(visual.astype(np.int16) - np.mean(visual))
    
    w, h = visual3.shape[:2]
    acc = []
    for i in range(h):
        rr, cc = line(0, i, w-1, h-i-1)
        acc.append(np.sum(img[rr, cc]))
    mi = np.argmax(acc)
    ret = np.zeros(img.shape[:2], dtype=np.bool_)
    rr, cc = line(0, mi, w-1, h-mi-1)
    ret[rr, cc] = True

    ret = morphology.binary_dilation(ret)
    ret = morphology.binary_dilation(ret)
    ret = morphology.binary_dilation(ret)
    ret = morphology.binary_dilation(ret)
    ret = morphology.binary_dilation(ret)
    w, h = ret.shape
    ret[w//2-3:w//2+3, h//2-3:h//2+3] = False

    delta = np.mean(visual[ret]) - np.mean(visual)
    visual_blured = ndimage.gaussian_filter(visual, sigma=5)
    visual[ret] = visual_blured[ret]
    newmagsshift = np.exp(visual)
    newffts = newmagsshift * np.exp(1j*angles)
    newfft = np.fft.ifftshift(newffts)
    imrev = np.fft.ifft2(newfft)
    newim2 = np.abs(imrev).astype(np.uint8)
    gaussian_blurred = cv2.GaussianBlur(newim2, (5, 5), 0)

    img = cv2.Canny(gaussian_blurred, 40, 50, apertureSize=3)

    ret, labels = cv2.connectedComponents(img, connectivity=8)
    # Map component labels to hue val
    label_hue = np.uint8(179*labels/np.max(labels))
    blank_ch = 255*np.ones_like(label_hue)
    labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])

    # Convert to BGR for display
    labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)

    # Set bg label to black
    labeled_img[label_hue == 0] = 0
    
    return labeled_img


In [3]:
image1 = cv2.imread('/home/pc-mint/Изображения/New Folder With Items/photo_2024-10-03 15.39.36.jpeg')

In [4]:
image2 = cv2.imread('/home/pc-mint/Изображения/New Folder With Items/photo_2024-10-03 15.39.33.jpeg')

In [5]:
image3 = cv2.imread('/home/pc-mint/Изображения/New Folder With Items/photo_2024-10-03 15.39.39.jpeg')

In [6]:
image1 = page_border(image1)

In [7]:
image2 = page_border(image2)

In [8]:
#image3 = page_border(image3)

In [9]:
cv2.imshow('Image2', image1)
cv2.waitKey(0)
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x1b129f0) is not the object's thread (0x20cfb00).
Cannot move to target thread (0x1b129f0)

QObject::moveToThread: Current thread (0x1b129f0) is not the object's thread (0x20cfb00).
Cannot move to target thread (0x1b129f0)

QObject::moveToThread: Current thread (0x1b129f0) is not the object's thread (0x20cfb00).
Cannot move to target thread (0x1b129f0)

QObject::moveToThread: Current thread (0x1b129f0) is not the object's thread (0x20cfb00).
Cannot move to target thread (0x1b129f0)

QObject::moveToThread: Current thread (0x1b129f0) is not the object's thread (0x20cfb00).
Cannot move to target thread (0x1b129f0)

QObject::moveToThread: Current thread (0x1b129f0) is not the object's thread (0x20cfb00).
Cannot move to target thread (0x1b129f0)

QObject::moveToThread: Current thread (0x1b129f0) is not the object's thread (0x20cfb00).
Cannot move to target thread (0x1b129f0)

QObject::moveToThread: Current thread (0x1b129f0) is not the object's thread

In [10]:
cv2.imshow('Image2', image2)
cv2.waitKey(0)
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x1b129f0) is not the object's thread (0x20cfb00).
Cannot move to target thread (0x1b129f0)

QObject::moveToThread: Current thread (0x1b129f0) is not the object's thread (0x20cfb00).
Cannot move to target thread (0x1b129f0)

QObject::moveToThread: Current thread (0x1b129f0) is not the object's thread (0x20cfb00).
Cannot move to target thread (0x1b129f0)

QObject::moveToThread: Current thread (0x1b129f0) is not the object's thread (0x20cfb00).
Cannot move to target thread (0x1b129f0)

QObject::moveToThread: Current thread (0x1b129f0) is not the object's thread (0x20cfb00).
Cannot move to target thread (0x1b129f0)

QObject::moveToThread: Current thread (0x1b129f0) is not the object's thread (0x20cfb00).
Cannot move to target thread (0x1b129f0)

QObject::moveToThread: Current thread (0x1b129f0) is not the object's thread (0x20cfb00).
Cannot move to target thread (0x1b129f0)

QObject::moveToThread: Current thread (0x1b129f0) is not the object's thread

In [11]:
#cv2.imshow('Image2', image3)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x1b129f0) is not the object's thread (0x20cfb00).
Cannot move to target thread (0x1b129f0)

QObject::moveToThread: Current thread (0x1b129f0) is not the object's thread (0x20cfb00).
Cannot move to target thread (0x1b129f0)

QObject::moveToThread: Current thread (0x1b129f0) is not the object's thread (0x20cfb00).
Cannot move to target thread (0x1b129f0)

QObject::moveToThread: Current thread (0x1b129f0) is not the object's thread (0x20cfb00).
Cannot move to target thread (0x1b129f0)

QObject::moveToThread: Current thread (0x1b129f0) is not the object's thread (0x20cfb00).
Cannot move to target thread (0x1b129f0)

QObject::moveToThread: Current thread (0x1b129f0) is not the object's thread (0x20cfb00).
Cannot move to target thread (0x1b129f0)

QObject::moveToThread: Current thread (0x1b129f0) is not the object's thread (0x20cfb00).
Cannot move to target thread (0x1b129f0)

QObject::moveToThread: Current thread (0x1b129f0) is not the object's thread